# MaxMinPicker

Notebook that investigates the improvements to MaxMinPicker that will be present in the 2017_09 RDkit release.
This work is described in Roger Sayle's talk at the 
[2017 UGM](https://github.com/rdkit/UGM_2017/blob/master/Presentations/Sayle_RDKitDiversity_Berlin17.pdf).

In [1]:
from rdkit import Chem
from rdkit.Chem import Draw,rdMolDescriptors,AllChem
from rdkit import SimDivFilters,DataStructs
import gzip, time, platform

In [2]:
print('Python Version', platform.python_version())
print('RDKit Version:', Chem.rdBase.rdkitVersion)

Python Version 3.6.1
RDKit Version: 2017.09.1


We'll use Andrew Dalke's [benzodiazepine dataset](http://dalkescientific.com/writings/benzodiazepine.sdf.gz) as its drug like and of a useful size for these studies. Download the file and name it `benzodiazepine.sdf.gz`.
We need to generate fingerprints for all molecules.

In [3]:
start = time.time()
benzodiazepines = []
inf = gzip.open('../data/benzodiazepine.sdf.gz')
suppl = Chem.ForwardSDMolSupplier(inf)
for mol in suppl:
    if mol is None: next
    benzodiazepines.append(rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2))    
inf.close()
end = time.time()
print('Read', len(benzodiazepines), 'molecules in', str(end - start)+' sec')

Read 12386 molecules in 13.598247051239014 sec


Now let's get picking.

In [4]:
start_with = 100
how_many_to_pick = 100
mmp = SimDivFilters.MaxMinPicker()
for i in 1,2,3:
    start = time.time()
    picks = mmp.LazyBitVectorPick(benzodiazepines, len(benzodiazepines), start_with + how_many_to_pick, list(range(start_with)))
    end = time.time()
    print('Picking', how_many_to_pick, 'from', len(benzodiazepines) - start_with, 'starting with', start_with, 'generated', len(picks) - start_with, 'picks')
    print('Picking took', '%.2f'%(end - start)+' sec')

Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 0.22 sec
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 0.21 sec
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 0.22 sec


Here's the output for the `2017.03.3` version of the RDKit run on a laptop:
```
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 32.70 sec
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 31.74 sec
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 31.85 sec
```
and the results for the `2017.09.1` version on the same laptop:
```
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 0.22 sec
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 0.21 sec
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 0.22 sec
```

That's more than 100x faster than the old algorithm for picking 100 given 100 seeds, faster still for larger sets (you'll need to run with different versions of RDKit to check this).
You can investigate the timings yourself by changing the `start_with` and `how_many_to_pick` variables.

Let's see how it can work on a real example. We'll take the [NCI250 data](https://cactus.nci.nih.gov/download/nci/) set with ~250K molecules as our starting set (assume these are the compounds we already have in our collection) and we want to pick 500 molecules from the benzodiazepine dataset to add to that set.

So let's read in the NCI dataset

In [5]:
start = time.time()
nciFps = []
suppl2 = Chem.SmilesMolSupplier('../data/NCISMA99.sdz', delimiter=' ', smilesColumn=1)
for mol in suppl2:
    if mol is None: continue
    nciFps.append(rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2))    
end = time.time()
print('Read', len(nciFps), 'molecules in', str(end - start)+' sec')

Read 247477 molecules in 97.62385106086731 sec


Let's look at how long it takes to pick 1000 diverse compounds from this set:

In [7]:
how_many_to_pick = 1000
mmp = SimDivFilters.MaxMinPicker()
for i in 1,2,3:
    start = time.time()
    picks = mmp.LazyBitVectorPick(nciFps, len(nciFps), 1000)
    end = time.time()
    print('Picking', how_many_to_pick, 'from', len(nciFps), 'generated', len(picks), 'picks')
    print('Picking took %.2f sec'%(end - start))

Picking 1000 from 247477 generated 1000 picks
Picking took 22.68 sec
Picking 1000 from 247477 generated 1000 picks
Picking took 21.88 sec
Picking 1000 from 247477 generated 1000 picks
Picking took 22.67 sec


Output of that for the `2017.09.1` version on the laptop:
```
Picking 1000 from 247477 generated 1000 picks
Picking took 22.68 sec
Picking 1000 from 247477 generated 1000 picks
Picking took 21.88 sec
Picking 1000 from 247477 generated 1000 picks
Picking took 22.67 sec
```

Now combine the nci and benzodiazepine fingerprints into a single list

In [8]:
allFps = []
allFps.extend(nciFps)
allFps.extend(benzodiazepines)
len(allFps)

259863

In [9]:
how_many_to_pick = 1000
seed_count = len(nciFps)
seeds = list(range(seed_count))
fp_num = len(allFps)
start = time.time()
picks = mmp.LazyBitVectorPick(allFps, fp_num, seed_count + how_many_to_pick, seeds)
end = time.time()
print('Picking', how_many_to_pick, 'from', len(allFps), 'starting with', seed_count, 'generated', len(picks) - seed_count, 'picks and took', str(end - start)+' sec')

Picking 1000 from 259863 starting with 247477 generated 1000 picks and took 1757.7510120868683 sec


The timing for that on the same laptop used above is:
```
Picking 1000 from 259863 starting with 247477 generated 1000 picks and took 1757.7510120868683 sec
```

So that's a reasonably representative selection of 1000 compounds done in about 20 minutes on a modestly speced laptop.
Not bad! You certainly couldn't do that with the old algorithm.

Now, let's look at a different approach. Instead of choosing how many to pick we specify a similarity threshold and keep picking until we have picked all the available molecules that are at least that distance from any that have already been picked. This uses the new function LazyBitVectorPickWithThreshold which returns the picks and the threshold of the last pick.

In [11]:
start = time.time()
picks, thresh = mmp.LazyBitVectorPickWithThreshold(benzodiazepines, len(benzodiazepines), 10000, 0.7)
end = time.time()
print('Picking generated', len(picks), 'picks, final threshold was', thresh)
print('Picking took %.2f sec'%(end - start))

Picking generated 146 picks, final threshold was 0.7
Picking took 0.16 sec


One thing to note here is that the last parameter seems to be a **disimilarity** score which is a bit unexpected. With a value of 0.7 you get around 146 picks, but with 0.4 you get 2065 indicating a closer placement of the picks:

In [12]:
start = time.time()
picks, thresh = mmp.LazyBitVectorPickWithThreshold(benzodiazepines, len(benzodiazepines), 10000, 0.4)
end = time.time()
print('Picking generated', len(picks), 'picks, final threshold was', thresh)
print('Picking took %.2f sec'%(end - start))

Picking generated 2065 picks, final threshold was 0.4
Picking took 5.26 sec


Note that in both cases we specify a stupidly high value for the number to pick so that this is not limiting.
The picker terminates once it can find no more to pick.
Of course you can specify a lower limit here so that it terminates when it can find more more to pick or has picked the required number.

Adios. Hats off to Roger and Greg!

# An aside

If you aren't working with fingerprints, there's another function, `LazyPick()`, that can be used to pick diverse compounds. This allows to pass in a function that generates the distance between the molecules.
This is a lot slower than LazyBitVectorPick as it does more too and fro between the C++ and Python layers, but does not assume you want the Tanimoto distance between 2 bit vectors, allowing you to speficy the distance function.

Here we'll show an example of using the LazyPick function by passing in a function that calculates the Tamimoto distance between the fingerprints (the same as LazyBitVectorPick).

The function:

In [13]:
def fn(i,j,fps=benzodiazepines):
    return 1.-DataStructs.TanimotoSimilarity(fps[i],fps[j])

LazyPick parameters:
1. the function that's used to generate the distance between 2 mols
2. the total number of molecules
3. the total number of molcules to return (including the initial seed molecules)
4. the indexes of the initial seed molecules (optional) e.g. [1,2,3,4 ...]

What's returned is the IDs of the initial seed molecules followed by the newly picked molecules

In [14]:
mmp = SimDivFilters.MaxMinPicker()
start_with = 100
how_many_to_pick = 100
for i in 1,2,3:
    start = time.time()
    picks = mmp.LazyPick(fn, len(benzodiazepines), start_with + how_many_to_pick, list(range(start_with)))
    end = time.time()
    print('Picking', how_many_to_pick, 'from', len(benzodiazepines) - start_with, 'starting with', start_with, 'generated', len(picks) - start_with, 'picks')
    print('Picking took %.2f sec'%(end - start))

Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 1.22 sec
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 1.18 sec
Picking 100 from 12286 starting with 100 generated 100 picks
Picking took 1.21 sec


Notice that this takes considerably longer than using `LazyBitVectorPick()`; it's definitely better to use that when possible.